In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import re

import ipywidgets as widgets
from ipywidgets import GridBox, Button
from IPython.display import display, clear_output

import os
from datetime import datetime

C:\Users\power\AppData\Local\Temp\ipykernel_14504\1987386995.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


### Read progress and records files

In [2]:
environments = {
    "water_tank": {
        "base_dir": "../logs/ppo/double_water_tank/trainings",
        "tracked_point": "x2",
    },
    "CPAP": {
        "base_dir": "../logs/ppo/CPAP",
        "tracked_point": "x3",
    },
}


env = environments["water_tank"]
base_dir = env["base_dir"]
tracked_point = env["tracked_point"]

index = None # -1
folder_names = [f for f in os.listdir(base_dir) if os.path.isdir(os.path.join(base_dir, f))][:index]
def folder_to_datetime(folder_name):
    """
    Função para converter o nome da pasta em um objeto datetime.
    Combinada com max(), é possível ordenar de forma decrescente pastas por data da mais recente (max) para antiga (min).
    O nome da pasta DEVE estar no formato '%d-%m-%H%M' ou ter uma substring nesse formato para funcionar corretamente.
    """
    pattern = r'\d{2}-\d{2}-\d{4}'
    match = re.search(pattern, folder_name)
    if match:
        date_str = match.group(0)
        date_str = datetime.strptime(date_str, '%d-%m-%H%M')
        return (date_str.month, date_str.day)
    else:
        raise ValueError(f"Formato de data inválido na pasta: {folder_name}")


folder_names.sort(key=folder_to_datetime)

latest_folder = folder_names[-1] # max(folder_names, key=folder_to_datetime)
selected_folder = latest_folder
selected_index = len(folder_names) - 1


# print(folder_names)
# print(f"A pasta mais recente é: {latest_folder}")

In [3]:
def are_required_files_present(folder: str):
    return all([
        os.path.exists(f"{base_dir}/{folder}/progress.csv"),
        os.path.exists(f"{base_dir}/{folder}/records.csv"),
        # os.path.exists(f"{base_dir}/{folder}/only_pid_records.csv"),
    ])

In [4]:
def load_dfs(folder: str) -> tuple[pd.DataFrame, pd.DataFrame, pd.DataFrame]:

    if not are_required_files_present(folder):
        raise FileNotFoundError("Arquivos necessários não encontrados.")
    
    progress_df = pd.read_csv(f"{base_dir}/{folder}/progress.csv")
    rec_df = pd.read_csv(f"{base_dir}/{folder}/records.csv")

    # Find the index of the last episode start
    last_episode_start_idx = rec_df[rec_df['is_start_episode']].index[-1]

    # Filter the dataframe for the last episode
    last_episode_df = rec_df.loc[last_episode_start_idx:]

    return progress_df, rec_df, last_episode_df

progress_df, rec_df, last_episode_df = load_dfs(latest_folder)

In [5]:
progress_df.head()

,train/clip_fraction,train/entropy_loss,train/value_loss,train/loss,train/n_updates,train/policy_gradient_loss,train/iteration,train/explained_variance,train/approx_kl,train/std,train/clip_range,train/learning_rate
0,0.0,-13.471673,1.386667e-10,-0.26949,20000,0.0,8,NaN,0.000015,172108.203125,0.2,0.0003


In [6]:
rec_df

,x1,x2,y_ref,z_t,PID_action,PPO_action,combined_action,reward,error,steps_in_episode,is_start_episode
0,1.883883,8.754544,3.0,0.000000,-50.639988,1.000000,-49.639988,-29.959132,-5.473494,1,True
1,1.640698,8.473494,3.0,-5.473494,-48.166744,0.155973,-48.010770,-26.880995,-5.184688,2,False
2,1.413751,8.184689,3.0,-10.658182,-45.625259,-0.977838,-46.603097,-23.897128,-4.888469,3,False
3,1.203084,7.888469,3.0,-15.546651,-43.018526,0.849241,-42.169286,-21.023854,-4.585178,4,False
4,1.008747,7.585177,3.0,-20.131828,-40.349563,-0.325771,-40.675334,-18.276989,-4.275160,5,False
...,...,...,...,...,...,...,...,...,...,...,...
399995,1.813938,1.906666,2.0,25.000000,0.821337,1.000000,1.821337,-0.009872,0.099357,1996,False
399996,1.939578,1.900643,2.0,25.000000,0.874342,-1.000000,-0.125658,-0.009383,0.096868,1997,False
399997,1.692825,1.903132,2.0,25.000000,0.852437,-1.000000,-0.147563,-0.012270,0.110768,1998,False
399998,1.462301,1.889232,2.0,25.000000,0.974760,-1.000000,-0.025240,-0.019612,0.140045,1999,False


### Plot Train over iteraçtion

In [7]:
def plot_progress(folder: str, save = False):
    # Plotting train/loss vs train/iteration
    plt.figure(figsize=(12, 6))
    plt.plot(progress_df['train/iteration'], progress_df['train/loss'], label='Train Loss')
    plt.xlabel('Iteration')
    plt.ylabel('Loss')
    plt.title('Train Loss vs. Iteration')
    plt.legend()
    plt.grid(True)
    if save:
        plt.savefig(f"{base_dir}/{folder}/loss_over_iteration.png")
    plt.show()

# plot_progress(latest_folder, save = True)

### Plote reward over steps

In [8]:
def plot_rewards(folder: str, save=False):
    # Plotting reward vs steps
    plt.figure(figsize=(12, 6))
    plt.plot(last_episode_df['steps_in_episode'], last_episode_df['reward'], label='Reward')
    plt.xlabel('Steps')
    plt.ylabel('Reward')
    plt.title('Reward vs. Steps')
    plt.legend()
    plt.grid(True)
    if save:
        plt.savefig(f"{base_dir}/{folder}/reward_over_last_episode_steps.png")
    plt.show()

# plot_rewards(latest_folder, save=True)

### Plot Set point curve

In [9]:
def plot_tracking(folder: str, save: bool = False):
    # Plot y_ref vs steps_in_episode
    plt.figure(figsize=(12, 6))
    plt.plot(last_episode_df['steps_in_episode'], last_episode_df[tracked_point], label="Altura da água no segundo tanque", color='red')
    plt.plot(last_episode_df['steps_in_episode'], last_episode_df['y_ref'], '--', label='y_ref (Objetivo)', color='black')
    plt.xlabel('Steps')
    plt.ylabel('')
    plt.title('Altura da água no segundo tanque em cada passo no último episódio')
    plt.legend()
    plt.grid(True)
    if save:
        plt.savefig(f"{base_dir}/{folder}/tracking.png")
    plt.show()

# plot_tracking(latest_folder, save=True)

### Ação do PPO e PID durante o último episódio

In [10]:
def plot_action_per_steps(folder: str, save: bool = False, separeted: bool = True):

    if (separeted):
        # Plotting action vs steps in the last episode
        plt.figure(figsize=(12, 6))
        plt.plot(last_episode_df['steps_in_episode'], last_episode_df['PID_action'], label='PID_action', color='blue')
        plt.xlabel('Steps')
        plt.ylabel('Action')
        plt.title('Ação do controlador PID em cada passo no último episódio')
        plt.legend()
        plt.grid(True)
        if save:
            plt.savefig(f"{base_dir}/{folder}/PID_action.png")
        plt.show()

        # Plotting action vs steps in the last episode
        plt.figure(figsize=(12, 6))
        plt.plot(last_episode_df['steps_in_episode'], last_episode_df['PPO_action'], label='PPO_action', color='red')
        plt.xlabel('Steps')
        plt.ylabel('Action')
        plt.title('Ação do controlador PPO em cada passo no último episódio')
        plt.legend()
        plt.grid(True)
        if save:
            plt.savefig(f"{base_dir}/{folder}/PPO_action.png")
        plt.show()
    else:
        # Plotting action vs steps in the last episode
        plt.figure(figsize=(12, 6))
        plt.plot(last_episode_df['steps_in_episode'], last_episode_df['PID_action'], label='PID_action', color='blue')
        plt.plot(last_episode_df['steps_in_episode'], last_episode_df['PPO_action'], label='PPO_action', color='red')
        # plt.plot(last_episode_df['steps_in_episode'], last_episode_df['action'], label='Action', color='black')
        plt.xlabel('Steps')
        plt.ylabel('Action')
        plt.title('Ação em cada passo no último episódio')
        plt.legend()
        plt.grid(True)
        if save:
            plt.savefig(f"{base_dir}/{folder}/combined_action.png")
        plt.show()

# plot_action_per_steps(latest_folder, save=True)

# Função para exibir a grade de pastas

In [11]:
def display_folders_grid(folder_names: list[str], 
                         selected_indices: list[int] = 0
                         ) -> GridBox:
    num_cols = 5
    # num_rows = (len(folder_names) + num_cols - 1) // num_cols
    previous_selected = [selected_indices]
    
    def on_button_click(b: Button):
        global selected_folder
        selected_folder = b.description

        global progress_df, rec_df, last_episode_df
        progress_df, rec_df, last_episode_df = load_dfs(selected_folder)

        previous_selected.append(buttons.index(b))
        if len(previous_selected) > 2:
            previous_selected.pop(0)
        
        # Atualiza cor dos botões
        buttons[previous_selected[0]].button_style = '' 
        buttons[previous_selected[1]].button_style = 'success'

        clear_output(wait=True)  # Limpar as saídas anteriores
        display(grid)            # Reexibir a grade de botões

        plot_tracking(selected_folder, save=False)
        plot_rewards(selected_folder, save=False)
        plot_action_per_steps(selected_folder, save=False)
        # plot_progress(selected_folder, save=False)
    
    # Criar os botões
    buttons = []
    for i, folder in enumerate(folder_names):
        button = widgets.Button(description=folder, 
                                layout=widgets.Layout(width='auto', height='auto'), 
                                index=i
                                )
        button.on_click(on_button_click)
        
        buttons.append(button)
    
    buttons[selected_indices].button_style = 'success'  # Definir a cor do botão selecionado como verde
    
    grid = widgets.GridBox(
        children=buttons,
        layout=widgets.Layout(grid_template_columns=f'repeat({num_cols}, 1fr)')
    )
    display(grid)

    # on_button_click(buttons[selected_indices])
    return grid

# display_folders_grid(folder_names, selected_index)